In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import pywt
from torch.utils.data import DataLoader, Dataset
import torch
import pickle

In [2]:
def file_to_DataDrame(path):
    """
    This function takes in a file path and returns a dataframe with the data and the target values
    format:
        Fc5	        Fc3	        Fc1	        ...	Oz	        O2	        Iz	        target
    0	-0.000046	-0.000041	-0.000032	...	0.000040	0.000108	0.000055	0
    1	-0.000054	-0.000048	-0.000034	...	0.000064	0.000114	0.000074	0
    ...
    """

    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "") 
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    return df

def save_to_pickle(data, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data



In [3]:
import os

In [4]:
def read_all_file_df(num_exp=[3, 4], num_people=10):
    """    condct all files in one dataframe"""
    all_df = pd.DataFrame()
    for subject in range(1, num_people):
        for file in num_exp:
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            df = file_to_DataDrame(fileName)
            all_df = pd.concat([all_df, df], axis=0)
    return all_df


In [6]:
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

In [10]:

df = read_all_file_df([3,7,11],109)



In [11]:
df

In [18]:
signal=df.iloc[:,:-1].values

In [19]:
signal.shape


In [21]:
time= np.linspace(0, len(signal)/160, len(signal))
widths = np.geomspace(1, 1024, num=4)
sampling_period = sampling_period = np.diff(time).mean()
cwtmatr, freqs =pywt.cwt(signal, widths,'cgau4', sampling_period=sampling_period)
cwtmatr= torch.tensor(cwtmatr)


In [ ]:
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import Dataset
import os

class MyImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = []
        self.labels = []

        for label, label_dir in enumerate(sorted(os.listdir(directory))):
            for image_name in os.listdir(os.path.join(directory, label_dir)):
                self.images.append(os.path.join(directory, label_dir, image_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = read_image(image_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
cwt_df = 

In [ ]:
class CWTDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target
    
    # def transform(self, signal):
    #     time= np.linspace(0, len(signal)/160, len(signal))
    #     widths = np.geomspace(1, 1024, num=10)
    #     sampling_period = sampling_period = np.diff(time).mean()
    #     cwtmatr, _ = pywt.cwt(signal, widths,'cgau4', sampling_period=sampling_period)
    #     return cwtmatr

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # data = self.transform(self.data[idx-640:idx])
        return torch.tensor(self.data[idx]), torch.tensor(self.target[idx])

In [ ]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Pełnopołączona warstwa

    def forward(self, x):
        # Inicjalizacja ukrytych stanów i stanu komórek
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # wyjście ma rozmiar (batch_size, sequence_length, hidden_size)
        
        # Dekodowanie ukrytego stanu ostatniej czasowej sekwencji
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU jest dostępne. Używanie GPU...")
else:
    device = torch.device("cpu")
    print("GPU nie jest dostępne. Używanie CPU...")

In [ ]:
from torch.utils.data import DataLoader


In [ ]:
dataset = CWTDataset(df.iloc[:, :-1].values, df.iloc[:, -1].values)
x = dataset.__getitem__(0)
x
DataLoader = DataLoader(dataset, batch_size=10, shuffle=True)

In [ ]:
# Parametry modelu i danych
input_size = 64  # Liczba cech wejściowych
hidden_size = 128  # Rozmiar ukrytej warstwy LSTM
num_layers = 4  # Liczba warstw LSTM
num_classes = 3  # Liczba klas wyjściowych

model = SimpleLSTM(input_size, hidden_size, num_layers, num_classes)

# Funkcja straty i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Dla przykładu, załóżmy, że `dataloader` jest już zdefiniowany i przygotowany

num_epochs = 5  # Liczba epok

for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(DataLoader):
        # Przesyłanie danych i etykiet na właściwe urządzenie
        sequences = sequences.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward i optymalizacja
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(DataLoader)}], Loss: {loss.item():.4f}')


In [ ]:

time = np.linspace(0, 4000 / 160, 4000)  # time of the signal
signal = fc5
widths = np.geomspace(1, 1024, num=100)  # range of scales
sampling_period = np.diff(time).mean()  # 0.006251562890722681
print(sampling_period)
wavlist = pywt.wavelist(kind="continuous")
for wavelet in wavlist:
    # A few wavelet families require parameters in the string name
    if wavelet in ["cmor", "shan"]:
        wavelet += "1-1"
    elif wavelet == "fbsp":
        wavelet += "1-1.5-1.0"

    # compute the wavelet transform
    cwtmatr, freqs = pywt.cwt(signal, widths, wavelet, sampling_period=sampling_period)
    # cwtmatr= np.abs(cwtmatr[:-1,:-1])
    cwtmatr = np.abs(cwtmatr)
    # plot the wavelet transform
    plt.figure(figsize=(20, 3))
    plt.pcolormesh(time, freqs, cwtmatr)

    maxval = np.max(freqs)
    plt.vlines(x=xT0, ymin=0, ymax=maxval, color="r", label="T0", linestyles="dashed")
    plt.vlines(x=xT1, ymin=0, ymax=maxval, color="c", label="T1", linestyles="dashed")
    plt.vlines(x=xT2, ymin=0, ymax=maxval, color="y", label="T2", linestyles="dashed")
    plt.yscale("log")
    plt.ylabel("Frequency [Hz]")
    plt.xlabel("Time [s]")
    plt.title(f"Wavelet Transform with {wavelet}")
    plt.colorbar()
    plt.show()

In [ ]:


def file_to_tensor(path):


    reader = mne.io.read_raw_edf(path, preload=True)
    annotations = reader.annotations  # get the values of the annotations
    codes = annotations.description  # get the codes from the annotations

    df = pd.DataFrame(
        reader.get_data().T,
        columns=[channel.replace(".", "") for channel in reader.ch_names],
    )  # transpose the data to get the right shape
    df = df[~(df == 0).all(axis=1)]  # remove rows with all zeros
    timeArray = np.array(
        [round(x, 10) for x in np.arange(0, len(df) / 160, 0.00625)]
    )  # create an array of time values

    codeArray = []
    counter = 0
    for timeVal in timeArray:
        if (
            timeVal in annotations.onset
        ):  # if the time value is in the onset array, add the corresponding code to the codeArray
            counter += 1
        code_of_target = int(
            codes[counter - 1].replace("T", "")
        )  # convert T0 to 0, T1 to 1, etc
        codeArray.append(code_of_target)

    df["target"] = np.array(codeArray).T
    
    # Convert DataFrame to PyTorch tensors
    signals = torch.tensor(df.drop(columns=['target']).values).float()
    targets = torch.tensor(df['target'].values).long()
    
    return signals, targets


In [ ]:
subject = 1
file = 5
fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"

signals, targets = file_to_tensor(fileName)

In [ ]:
print(signals.shape, targets.shape)

In [ ]:
def read_all_files_tensor():
    all_signals = []
    all_targets = []
    for subject in range(1, 110):
        for file in range(1, 15):
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            signals, targets = file_to_tensor(fileName)
            all_signals.append(signals)
            all_targets.append(targets)
    return torch.cat(all_signals), torch.cat(all_targets)

In [ ]:
import pickle

def save_tensors_to_pickle(X, y, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump((X, y), f)

def load_tensors_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        X, y = pickle.load(f)
    return X, y

### What is done in experiment

1. Baseline, eyes open
2. Baseline, eyes closed
3. Task 1 (open and close left or right fist)
4. Task 2 (imagine opening and closing left or right fist)
5. Task 3 (open and close both fists or both feet)
6. Task 4 (imagine opening and closing both fists or both feet)
7. Task 1
8. Task 2
9. Task 3
10. Task 4
11. Task 1
12. Task 2
13. Task 3
14. Task 4

In [ ]:
def read_all_file_batch(batch_size=640, num_exp=[3, 4], num_people=10):
    all_signals = torch.empty(num_people, len(num_exp), 20000, batch_size, 64)
    all_targets = torch.empty(num_people, len(num_exp), 20000)
    index_exp = 0
    index_people = 0
    for subject in range(1, num_people):
        for file in num_exp:
            fileName = f"files/S{subject:03d}/S{subject:03d}R{file:02d}.edf"
            signals, targets = file_to_tensor(fileName)
            var_signals = torch.empty(20000, batch_size, 64)
            var_targets = torch.empty(20000)
            for i in range(0, 20000):
                if i > batch_size:
                    if i >= len(signals):
                        var_signals[i - batch_size] = torch.full(
                            (batch_size, 64), float("nan")
                        )
                        var_targets[i - batch_size] = float("nan")
                    else:
                        var_signals[i - batch_size] = signals[i - batch_size : i]
                        var_targets[i - batch_size] = targets[i]
            all_signals[index_people][index_exp] = var_signals
            all_targets[index_people][index_exp] = var_targets
            index_exp += 1
        index_people += 1
    return all_signals, all_targets

In [ ]:
del signals, targets

In [ ]:
all_signals, all_targets = read_all_file_batch(640,[3,7,1],109)
save_tensors_to_pickle(all_signals, all_targets, 'all_signals_targets_batch.pickle')
del all_signals, all_targets

In [ ]:
all_signals, all_targets = read_all_files_tensor()

In [ ]:
import pickle

def save_tensors_to_pickle(X, y, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump((X, y), f)

def load_tensors_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        X, y = pickle.load(f)
    return X, y

In [ ]:
save_tensors_to_pickle(all_signals, all_targets, 'pkl/all_signals_targets.pkl')
del all_signals, all_targets

In [ ]:
all_signals, all_targets = load_tensors_from_pickle('pkl/all_signals_targets.pkl')

In [ ]:
all_signals.shape, all_targets.shape